# Advanced machine learning - 2nd Assignment

Per prima cosa impostiamo impostiamo il seed per le backend di Numpy e Keras in modo da ottenere risultati riproducibili

In [1]:
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)

Dunque importiamo il dataset come Dataframe Pandas, separato in fetures_set e labels_set. I dati sono recuperati direttamente da repository GitHub. 
Il dataset duqnue viene unito utilizzando come identificatore la colonna *ID*.

In [2]:
import pandas as pd 
from modules import utils
import numpy as np

features_url = 'https://raw.githubusercontent.com/AlbezJelt/AML_Assignment2/main/data/X_train.csv'
labels_url = 'https://raw.githubusercontent.com/AlbezJelt/AML_Assignment2/main/data/y_train.csv'
train_set = utils.import_training_dataset(features_url, labels_url)

In [3]:
import plotly.express as px
from sklearn.decomposition import PCA

x_train, y_train = utils.prepare_data(train_set)
x_train_standardized, scaler = utils.preprocess_data(x_train)
pca = PCA(n_components=2)
comp = pca.fit_transform(x_train_standardized)
fig = px.scatter(comp, x=0, y=1, color=y_train)
fig.show()

## Definizione dei modelli

Procediamo ora con la costruzione di **due diversi modelli, entrambi con la stessa struttura ma uno implementate tecniche di regolarizzazione**.

Il modello si compone di 1 input layer, 3 dense hidden layer ed 1 output layer.  
Ogni hidden layer è formato da 23 unità, equivalenti al numero di feature in input, ed utilizza *relu* come funzione di attivazione.  
L'output layer invece è una singola unità con funzione di attivazione *sigmoid*, adatta ad una classificazione di un output binario.  
La loss function ottimizzata è *BinaryCrossentropy*, specifica per problemi di classificazione binaria, con optimizer *Adam*, che mostra buoni risultati anche con i parametri di default.


In [4]:
import keras
from keras import layers
from sklearn import metrics
from tensorflow.keras import regularizers

def build_model(input_layer_dim):
    
    model = keras.Sequential()
    
    model.add(
        layers.Dense(
            units=23,
            activation='relu',
            input_shape=(input_layer_dim, )
        )
    )
    
    model.add(
        layers.Dense(
            units=23,
            activation='relu'
        )
    )
 
    model.add(
        layers.Dense(
            units=23,
            activation='relu'
        )
    )

    model.add(
        layers.Dense(
            units = 1,
            activation ='sigmoid'
        )
    )
    
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer='adam',
        metrics=[tf.keras.metrics.BinaryCrossentropy(), 'AUC']
    )
    
    return model

### Modello con regolarizzazione

Per quanto riguarda il modello con regolarizzazione sono stati aggiunti:
- **Dropout layers** con probabilità 0.5 per ogni hidden layer.
- **L1 regularization** con valore fisso 0.001. Valori più alti (nell'ordine di >=10<sup>-2</sup>) portavano ad un apprendimento estremamente lento o addirittura nullo (underfitting).
- **L2 regularization** con valore fisso 0.001

In [17]:
def build_model_with_regularization(input_layer_dim):
    
    model = keras.Sequential()

    model.add(
        layers.Dense(
            units=23,
            activation='relu',
            input_shape=(input_layer_dim, ),
            kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
        )
    )
    
    
    model.add(
        layers.Dropout(0.5)
    )

    model.add(
        layers.Dense(
            units=23,
            activation='relu',
            kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
        )
    )
    
    model.add(
        layers.Dropout(0.5)
    )

    model.add(
        layers.Dense(
            units=23,
            activation='relu',
            kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
        )
    )

    model.add(
        layers.Dense(
            units = 1,
            activation ='sigmoid'
        )
    )
    
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer='adam',
        metrics=[tf.keras.metrics.BinaryCrossentropy(), 'AUC']
    )
    
    return model

Di seguito vengono definite due nuove estensioni delle callback utilizzate da Keras:
- **F1History**: utilizzata per calcolare l'F1 score sul validation set alla fine di ogni epoca.
- **CustomStopper**: estensione della callback EarlyStopping che implementa la possibilità di attivare l'early stopping dopo un numero di epoche *start_epoch* in input. **Ciò permette di utilizzare tale metodologia di regolarizzazione anche su un modello che apprende lentamente** (a causa, ad esempio, di un alto valore di dropout) tenendo arbitrariamente (in funzione di *start_epoch*) basso il valore di *pazienza* e *delta*.

In [6]:
class F1History(tf.keras.callbacks.Callback):

    def __init__(self, validation):
        super(F1History, self).__init__()
        self.validation = validation

    def on_epoch_end(self, epoch, logs={}):
        logs['F1_score_val'] = float('-inf')
        X_valid, y_valid = self.validation[0], self.validation[1]
        y_val_pred = (self.model.predict(X_valid).ravel()>0.5).astype("int32")
        val_score = metrics.f1_score(y_valid, y_val_pred)
        logs['F1_score_val'] = np.round(val_score, 5)

class CustomStopper(keras.callbacks.EarlyStopping):
    
    def __init__(self, monitor='val_loss',
             min_delta=0, patience=0, verbose=0, mode='auto', restore_best_weights=True, start_epoch = 100): # add argument for starting epoch
        super(CustomStopper, self).__init__()
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.start_epoch:
            super().on_epoch_end(epoch, logs)        

Il dataset viene diviso in train e test set. Inoltre viene inizializzato un 5 shuffled fold, utilizzato durante la fase di training per validazione.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import callbacks
from sklearn.model_selection import KFold

X_t, X_test, y_t, y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

N_SPLITS = 5
sss = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

### Training del modello *SENZA* regolarizzazione

Il modello viene allenato e validato su ogni split generato via *KFold*, previa standardizzazione del training set. Viene salvato modello, history e scaler della rete che ottiene il miglior validation F1 score.

In [8]:
best_model = None
history_bm = None
scaler_bm = None

bFirst = True
for train_index, test_index in sss.split(X_t, y_t):

    x_val_fold, y_val_fold = X_t[test_index], y_t[test_index]
    x_train_fold, y_train_fold = X_t[train_index], y_t[train_index]

    # Standardize train set
    x_train_fold, scaler = utils.preprocess_data(x_train_fold)
    # Use the computed scaler to standardize validation set
    x_val_fold, sclaer = utils.preprocess_data(x_val_fold, scaler=scaler)

    model = build_model(x_train_fold.shape[1])

    history = model.fit(
        x_train_fold, 
        y_train_fold, 
        epochs=100, 
        batch_size=32, 
        verbose=0,
        validation_data=(x_val_fold,y_val_fold),
        callbacks=[
            callbacks.ProgbarLogger(count_mode='steps'), 
            F1History(validation=(x_val_fold,y_val_fold))
            ]
        )

    if bFirst:
        best_model = model
        history_bm = history
        scaler_bm = scaler
        bFirst = False
        print(f"val_loss: {history.history['val_loss'][-1]} - val_f1_score: {history.history['F1_score_val'][-1]}")
    else:
        if history.history['F1_score_val'][-1] > history_bm.history['F1_score_val'][-1]:
            best_model = model
            history_bm = history
            scaler_bm = scaler
            print(f"val_loss: {history.history['val_loss'][-1]} - val_f1_score: {history.history['F1_score_val'][-1]}")
        elif history.history['F1_score_val'][-1] == history_bm.history['F1_score_val'][-1]:
            if history.history['val_loss'][-1] < history_bm.history['val_loss'][-1]:
                best_model = model
                history_bm = history
                scaler_bm = scaler
                print(f"val_loss: {history.history['val_loss'][-1]} - val_f1_score: {history.history['F1_score_val'][-1]}")


val_loss: 0.5184562802314758 - val_f1_score: 0.45632
val_loss: 0.49027949571609497 - val_f1_score: 0.48513


### Training del modello *CON* regolarizzazione

Il modello viene allenato e validato su ogni split generato via *KFold*, previa standardizzazione del training set.  
Inoltre viene richiamata la callback CustomStopper per abilitare early stopping dopo 50 epoche di training.  
Viene salvato modello, history e scaler della rete che ottiene il miglior validation F1 score.

In [18]:
reg_best_model = None
reg_history_bm = None
reg_scaler_bm = None

bFirst = True
for train_index, test_index in sss.split(X_t, y_t):

    x_val_fold, y_val_fold = X_t[test_index], y_t[test_index]
    x_train_fold, y_train_fold = X_t[train_index], y_t[train_index]

    # Standardize train set
    x_train_fold, scaler = utils.preprocess_data(x_train_fold)
    # Use the computed scaler to standardize validation set
    x_val_fold, sclaer = utils.preprocess_data(x_val_fold, scaler=scaler)

    es = CustomStopper(
        patience=7, 
        verbose=0, 
        min_delta=0.001, 
        monitor='F1_score_val', 
        mode='max', 
        restore_best_weights=True,
        start_epoch = 50
        )

    model = build_model_with_regularization(x_train_fold.shape[1])

    history = model.fit(
        x_train_fold, 
        y_train_fold, 
        epochs=100, 
        batch_size=32, 
        verbose=0,
        validation_data=(x_val_fold,y_val_fold),
        callbacks=[
            callbacks.ProgbarLogger(count_mode='steps'), 
            F1History(validation=(x_val_fold,y_val_fold)),
            es
            ]
        )

    if bFirst:
        reg_best_model = model
        reg_history_bm = history
        reg_scaler_bm = scaler
        bFirst = False
        print(f"val_loss: {history.history['val_loss'][-1]} - val_f1_score: {history.history['F1_score_val'][-1]}")
    else:
        if history.history['F1_score_val'][-1] > reg_history_bm.history['F1_score_val'][-1]:
            reg_best_model = model
            reg_history_bm = history
            reg_scaler_bm = scaler
            print(f"val_loss: {history.history['val_loss'][-1]} - val_f1_score: {history.history['F1_score_val'][-1]}")
        elif history.history['F1_score_val'][-1] == reg_history_bm.history['F1_score_val'][-1]:
            if history.history['val_loss'][-1] < reg_history_bm.history['val_loss'][-1]:
                reg_best_model = model
                reg_history_bm = history
                reg_scaler_bm = scaler
                print(f"val_loss: {history.history['val_loss'][-1]} - val_f1_score: {history.history['F1_score_val'][-1]}")

val_loss: 0.4624112546443939 - val_f1_score: 0.51633
val_loss: 0.46409371495246887 - val_f1_score: 0.53593
val_loss: 0.459657222032547 - val_f1_score: 0.54112


Utilizzando la funzione describe di Pandas visualizziamo media e deviazione standard dei pesi delle varie unità per ogni layer:

In [44]:
for layer in best_model.layers:
    if type(layer) == keras.layers.core.Dense:
        print(layer.name)
        weight = layer.get_weights()[0]
        print(pd.DataFrame(weight).describe().loc[['mean', 'std']]) 

dense_4
            0         1         2         3         4         5         6   \
mean -0.015727  0.024240 -0.009024  0.021975  0.013009  0.080652  0.046294   
std   0.332724  0.289332  0.365989  0.325687  0.363355  0.281525  0.373339   

            7         8         9   ...        13        14       15  \
mean -0.015943  0.013423  0.049399  ...  0.059020 -0.102237  0.06298   
std   0.285297  0.392194  0.352082  ...  0.262723  0.356964  0.39508   

            16        17        18        19        20        21        22  
mean -0.122263  0.057002  0.152709  0.062903  0.047762 -0.046706 -0.071309  
std   0.355869  0.321407  0.273556  0.309222  0.312085  0.298127  0.336428  

[2 rows x 23 columns]
dense_5
            0         1         2         3         4         5         6   \
mean -0.127085 -0.023100 -0.081708 -0.073851 -0.036139  0.064964 -0.082946   
std   0.480399  0.393333  0.474373  0.408264  0.348235  0.323272  0.352505   

            7         8         9   ...    

Nel modello implementante regolarizzazione osserviamo come i pesi siano effettivamente più piccoli. In particolare a causa di regolarizzazione L1 i vettori sono molto più sparsi, con diverse medie e std che si avvicinano allo zero.  
Per l'output layer invece la situazione è simile in quanto non è stata applicata ne L1 ne L2.

In [42]:
for layer in reg_best_model.layers:
    if type(layer) == keras.layers.core.Dense:
        print(layer.name)
        weight = layer.get_weights()[0]
        print(pd.DataFrame(weight).describe().loc[['mean', 'std']])

dense_108
            0         1         2         3         4         5         6   \
mean -0.000001  0.000033  0.000025  0.018184  0.005084 -0.000030 -0.000018   
std   0.000115  0.000112  0.000135  0.068954  0.088576  0.000132  0.000121   

            7         8         9   ...        13        14        15  \
mean -0.000015 -0.000007  0.000023  ...  0.006291  0.000122  0.022205   
std   0.000138  0.000127  0.000100  ...  0.084560  0.000173  0.088964   

            16        17        18        19        20        21        22  
mean  0.000024 -0.000019 -0.000034  0.016409  0.000006  0.014284  0.029950  
std   0.000163  0.000121  0.000123  0.075817  0.000143  0.079028  0.079554  

[2 rows x 23 columns]
dense_109
            0         1         2         3         4         5         6   \
mean  0.000021 -0.000013  0.041158  0.000005 -0.000008 -0.000021 -0.000015   
std   0.000160  0.000089  0.061190  0.000101  0.000121  0.000129  0.000114   

            7         8         9   

**Il modello con regolarizzazione sembra comportarsi meglio**, producendo un F1 score maggiore:

In [48]:
# Validazione su testset del modello senza regolarizzazione
X_test_no_reg, scaler = utils.preprocess_data(X_test, scaler_bm)
y_val_pred_no_reg = (best_model.predict(X_test_no_reg).ravel()>0.5).astype("int32")
print(f"f1_score no reg: {metrics.f1_score(y_test, y_val_pred_no_reg)}")

# Validazione su testset del modello con regolarizzazione
X_test_reg, scaler = utils.preprocess_data(X_test, reg_scaler_bm)
y_val_pred_reg = (reg_best_model.predict(X_test_reg).ravel()>0.5).astype("int32")
print(f"f1_score with reg: {metrics.f1_score(y_test, y_val_pred_reg)}")

f1_score no reg: 0.4577572964669739
f1_score with reg: 0.5039883973894126


## Fase di predizione

Infine viene importato il dataset contenente le istanze da classificare. Ogni feature viene scalata utilizzando lo scaler associato al modello utilizzato ed ogni istanza viene classificata.  
I risultati vengono salvati in file di testo.

In [20]:
import os

to_predict = pd.read_csv('https://raw.githubusercontent.com/AlbezJelt/AML_Assignment2/main/data/X_test.csv')
to_predict.pop('ID')

x_pred = to_predict.copy().to_numpy()
x_pred, scaler = utils.preprocess_data(x_pred, reg_scaler_bm)

y_pred = (reg_best_model.predict(x_pred).ravel()>0.5).astype("int32")
pd.DataFrame(y_pred).to_csv(
    f"{os.getcwd()}/results/Federico_Alberici_808058_score2.txt",
    header=False,
    index=False
    )